# **Import**

In [1]:
import torch
import torchvision
from torch import nn  
import torchvision.datasets as datasets  
import torchvision.transforms as transforms  
from torch.utils.data import DataLoader  
from torch import optim  
from tqdm import tqdm  
import torch.nn.functional as F  


# **Custom class with inheritence**

In [2]:
class neuralNet(nn.Module):
    def __init__(self,input_size,num_classes):
        super(neuralNet,self).__init__()
        self.fc1= nn.Linear(input_size,50)
        self.fc2 =nn.Linear(50,num_classes)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x

# **setup device**

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **Hyperparameters**

In [4]:
input_size= 784
num_classes=10
learning_rate =0.001
batch_size=64
num_epochs=1

# **Dataset from standard pytorch datasets**

In [5]:
train_dataset=datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# **Instance of class neuralNet with device configuration**

In [6]:
model = neuralNet(input_size=input_size,num_classes=num_classes).to(device)

# **setup loss function and optimizer**

In [7]:
creterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

# **The famous Training loop**

In [8]:
for epoch in range(num_epochs):
    for batch_index, (data, targets) in enumerate(train_loader):
        #try to get data to cuda
        data=data.to(device=device)
        targets=targets.to(device=device)
        
        #shape
        data=data.reshape(data.shape[0],-1)
        
        #forward
        scores=model(data)
        loss=creterion(scores,targets)
        
        #backword
        optimizer.zero_grad()
        loss.backward()
        
        #gradient_descent
        optimizer.step()
        

# **Accuracy checking function**

In [9]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 93.31
Accuracy on test set: 93.13


In [10]:
check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

tensor(0.9313)